Analise de Filmes de arquivos .CSV

In [ ]:
import pyodbc
import pandas as pd
import datetime as dt
import pickle
from pyspark.sql import SparkSession,Row
from pyspark.sql.functions import *
from pyspark.sql import functions as f
from pyspark.sql.types import *


import os 
import os.path as path
import datetime as dt
import sys
#criando um objeto sparksession object e um appName 
spark=SparkSession.builder.master("local[1]")\
        .appName("movie")\
        .config("spark.driver.extraClassPath","/Users/eduardoalberto/opt/spark/jars/mssql-jdbc-7.4.1.jre8.jar" ) \
        .getOrCreate()

sc = spark.sparkContext
server = 'localhost'
database = 'DBDWP511'
username = 'sa'
password = 'Numsey@Password!'
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

In [ ]:
file = f"/Users/eduardoalberto/LoadFile/archive/credits.csv"
if path.exists(file):
# tratar do .csv
    arq = spark.read.option("delimiter", ",").option("header", True).option("inferSchema", True).option("escape",'"').csv(file)
# cast     
    arq.printSchema()
    schema = StructType([
            StructField("cast_id",      StringType(),True),
            StructField("character",    StringType(),True),
            StructField("credit_id",    StringType(),True),
            StructField("gender",       StringType(),True),
            StructField("id",           StringType(),True),
            StructField("name",         StringType(),True),
            StructField("order",        StringType(),True),
            StructField("profile_path", StringType(),True)
    ])
    df = arq.select(regexp_replace(regexp_replace(arq.cast,r"\[",""),r"\]","").alias("cast"))  
    df1 = df.withColumn("cast", from_json("cast", schema)).select(col('cast.*'))
    df1 = df1.withColumnRenamed("id", "id02")
# crew
    schema2 = StructType([
            StructField("credit_id",      StringType(),True),
            StructField("department",     StringType(),True),
            StructField("gender",         StringType(),True),
            StructField("id",             StringType(),True),
            StructField("job",            StringType(),True),
            StructField("name",           StringType(),True),
            StructField("profile_path",   StringType(),True)
    ])
    df2=  arq.select(regexp_replace(regexp_replace(arq.crew,r"\[",""),r"\]","").alias("crew"))
    df2 = df2.withColumn("crew", from_json("crew", schema2)).select(col("crew.*"))
    df2 = df2.withColumnRenamed("id", "id03")
# ID
    df3 = arq.select(arq.id)
    # junta tudo
    tbl_creditos = df1.join(df2, df1.id02 == df2.id03, "inner").join(df3, df3.id == df1.id02).select("*")

In [ ]:
file = f"/Users/eduardoalberto/LoadFile/archive/keywords.csv"
if path.exists(file):
    arq = spark.read.option("delimiter", ",").option("header", True).option("inferSchema", True).option("escape",'"').csv(file)
    df = spark.createDataFrame(arq.rdd)
    sch = StructType([
             StructField("id",       IntegerType(),True),
             StructField("name",     StringType(),True)])
    df = df.select(df.id,regexp_replace(regexp_replace(df.keywords,r"\[",""),r"\]","").alias("keywords"))
    tbl_keyworkds = df.withColumn("keywords", from_json("keywords", sch)).select(df.id.alias("id_key"),col("keywords.*"))

In [ ]:
file = f"/Users/eduardoalberto/LoadFile/archive/links_small.csv"
if path.exists(file):
    arq = spark.read.option("delimiter", ",").option("header", True).option("inferSchema", True).option("escape",'"').csv(file)
    tbl_links_small = arq

In [ ]:
file = f"/Users/eduardoalberto/LoadFile/archive/links.csv"
if path.exists(file):
    arq = spark.read.option("delimiter", ",").option("header", True).option("inferSchema", True).option("escape",'"').csv(file)
    tbl_links = arq
    

In [ ]:
from email import header
from operator import mod
from os import sep


file = f"/Users/eduardoalberto/LoadFile/archive/movies_metadata.csv"
parquet = "/Users/eduardoalberto/LoadFile/archive/parquet"
if path.exists(file):
    arq = spark.read.option("delimiter", ",").option("header", True).option("inferSchema", True).option("escape",'"').csv(file)
    # arq.printSchema()
    movie = arq.where(arq.adult == "False").select("*")
    movieAdult = arq.where(arq.adult == "True").select("*")
    
    sch01 = StructType([
            StructField("id",               IntegerType(),True),
            StructField("name",             StringType(),True),
            StructField("poster_path",      StringType(),True),
            StructField("backdrop_path",    StringType(),True)])

    movie_01 = movie.withColumn("belongs_to_collection", from_json("belongs_to_collection",sch01)). \
          select(arq.id.alias("id01"),
                 col("belongs_to_collection.*"),
                 arq.budget,
                 arq.imdb_id,
                 arq.original_language,
                 arq.original_title,
                 arq.overview,
                 arq.popularity,
                 arq.poster_path.alias("path_poster"),
                 arq.release_date.cast("date").alias("dt_release"),
                 arq.revenue,
                 arq.runtime,
                 arq.status,
                 arq.tagline,
                 arq.title,
                 arq.video,
                 arq.vote_average.cast("int").alias("vote_average"),
                 arq.vote_count.cast("int").alias("vote_count")
                 ) 
 
    sch02 = StructType([
            StructField("id",               IntegerType(),True),
            StructField("name",             StringType(),True)])
    
  # movie.select("genres").show(10, False)
    mv = movie.select(regexp_replace(regexp_replace(movie.genres,r"\[",""),r"\]","").alias("genres"))
    mv = mv.withColumn("genres", from_json("genres",sch02)).select(f.col("genres.*"))
    mv = mv.withColumnRenamed("id", "id02")
    mv = mv.withColumnRenamed("name", "name_geners")
    df_full = movie_01.join(mv, movie_01.id01 == mv.id02, "inner")

    df_full.write.parquet(path=parquet, mode="overwrite")

   #usando coalesce pra gerar arquivo unico
    df_full.coalesce(1).write.csv(path=parquet, mode="overwrite",sep=";",header=True)

    

    spark.stop
        
    

In [ ]:
porcent = lambda nulos, massa: (nulos / massa)*100
print((porcent(182562,1000000)))

data = [
    ('GISELLE PAULA GUIMARAES CASTRO', 15),
    ('ELAINE GARCIA DE OLIVEIRA', 22),
    ('JOAO CARLOS ABNER DE LOURDES', 43),
    ('MARTA ZELI FERREIRA', 24),
    ('LAUDENETE WIGGERS ROEDER', 51)
]
colNames = ['nome', 'idade']
df = spark.createDataFrame(data, colNames)
df.show(truncate=False)

df \
    .select(
        f.concat_ws(
            ', ', 
            f.substring_index('nome', ' ', 1), 
            f.substring_index('nome', ' ', -1)
        ).alias('ident'), 
        'idade') \
    .show(truncate=False)
